한국어의 띄어쓰기 교정 문제는 **본래 띄어써야 하는데 붙여쓴 경우를 교정하는 것**입니다. 한국어는 띄어쓰기를 잘 지키지 않아도 가독성에 큰 불편함에 없기에 띄어써야 하는 부분들을 붙여 씁니다. 특히 하나의 구 (phrase) 단위로 붙여 쓰는 경우들이 많습니다. 구 단위로는 띄어쓰기가 없어도 한 번에 의미를 파악하는데 큰 어려움이 없기 때문입니다.

| Noisy text | Clean text |
| --- | --- |
| 집에가자 | 집에 가자 |
| 이것도잘해보자 | 이것도 잘 해보자 |

붙여써야 하는데 띄어쓰는 경우는 상대적으로 매우 적으며, 실수라기 보다는 고의적으로 입력한 경우들이 많습니다. 아래와 같은 메시지는 강조의 의미로 굳이 띄어쓴 경우입니다. 띄어쓰기 교정기가 이런 경우까지 모두 고려하는 것은 문제의 폭을 지나치게 넓히는 것이기 때문에 오히려 잘못된 패턴을 학습할 가능성도 높습니다.

```
그 만 해 !!!!
```

띄어쓰기 교정기가 이용하는 정보는 한 글자의 앞/뒤에 등장하는 다른 글자들입니다. Conditional Random Field (CRF) 는 potential function 을 이용하여 앞/뒤에 등장하는 글자들의 정보를 features 로 이용합니다. 그러나 CRF 는 모든 경우에 동일한 features 를 이용하는 단점이 있습니다.

예를 들어 지금 시점 (i=0) 글자와 앞의 글자 (i=-1) 가 `X[-1:0] = 이다` 일 경우, 지금 글자 `다` 가 문장의 마지막 글자처럼 보일수 있습니다. CRF 는 `X[-1:0] = 이다` 라는 features 에 대하여 띄어쓸 점수 (coefficient) 를 크게 지닙니다. 하지만 문맥이 `X[-1:2] = 이다음에` 였다면 `이다 음에` 처럼 잘못 띄어질 가능성이 있습니다. 문맥에 따라서 features 를 골라서 이용해야 하는데, CRF 는 이러한 능력이 없습니다. 단, potential function 에 의하여 만들어진 high dimensional sparse Boolean vector space 에서 `이다음에`라는 문맥에 `X[-1:0] = 이다` 라는 feature 가 이용되더라도 띄어쓰기를 잘 할 수 있는 coefficient vectors 를 학습하려 노력합니다. 물론 대체로 CRF 의 학습 결과가 좋기는 하지만, 불명확한 features 는 상황에 따라 이용하지 않을 수 있으면 좋습니다. 그리고 `X[-1:0] = 이다` 는 문장의 끝 부분에 워낙 자주 등장하는 글자이기 때문에 `이다`가 포함된 단어는 띄어질 가능성이 높습니다. 즉 CRF 는 공격적으로 띄어쓰기를 교정할 가능성이 높습니다 (띄어쓰지 않아도 될 부분을 띄어쓸 가능성이 높습니다). 

soyspacing 은 CRF 처럼 앞/뒤에 등장하는 글자들을 features 로 이용하여 띄어쓰기를 교정하지만, `이다음에`와 같은 상황에서는 `X[-1:0] = 이다`를 feature 로 이용하지 않으며 띄어쓰기를 교정합니다.

또한 CRF 는 softmax 기준으로 0.01 이라도 더 높은 쪽의 띄어쓰기 label 을 선택합니다. 하지만 0.49 : 0.51 과 같은 확률을 지니는 애매모호한 상황에서는 왠만하면 띄어쓰기를 하지 않는 편이 좋습니다. 띄어쓰기 교정은 대체로 데이터의 전처리 과정에서 이용이 되는데, 혹여 띄어쓰기를 제대로 교정하지 않았다 하더라도 토크나이저를 적용하는 과정에서 나머지 오류가 교정될 수도 있기 때문입니다. 그러나 토크나이저들은 한 번 띄어진 단어들을 굳이 하나의 단어로 확인하지는 않습니다. 오히려 그 과정이 훨씬 많은 오류를 범할 수 있고, 그 비용도 비싸기 때문입니다. 즉, 한 번 잘못 띄어진 글자들은 이후로 제대로 인식될 가능성이 적습니다. 그렇기 때문에 애매모호한 상황에서는 차라리 띄어쓰기를 교정하지 않는 편이 좋습니다.

soyspacing 은 score threshold 를 두어, 특정 띄어쓰기 점수 이상이 될 때에만 띄어쓰기를 하며, 특정 붙여쓰기 점수 이하일 때에만 붙여쓰기를 합니다. 그 외에는 애매모호하기 때문에 '띄어쓰지 않기' 를 합니다.

현재 soyspacing 의 버전은 `soyspacing == 1.0.15` 이며, pip install 로 설치가 가능합니다.

```
pip install soyspacing
```

In [1]:
import config
import soyspacing
from navermovie_comments import load_movie_comments

print('soyspacing={}'.format(soyspacing.__version__))

idxs, texts, rates = load_movie_comments(idxs='134963')

soynlp=0.0.49
added lovit_textmining_dataset
soyspacing=1.0.15


soyspacing 은 input text 의 파일을 입력받아 학습합니다. 학습에 이용할 파일을 만듭니다. 영화 라라랜드의 리뷰 텍스트를 이용합니다.

In [2]:
corpus_path = 'lalaland_comments.txt'
with open('lalaland_comments.txt', 'w', encoding='utf-8') as f:
    for text in texts:
        f.write('{}\n'.format(text.strip()))

학습은 model 의 train 함수에 input text file 의 주소를 입력하면 됩니다.

In [3]:
from soyspacing.countbase import CountSpace

model = CountSpace()
model.train(corpus_path)

all tags length = 654328 --> 53317, (num_doc = 15598)

학습된 모델은 아래처럼 save / load 를 할 수 있습니다.

In [4]:
model_path = 'lalaland-soyspacing.model'
model.save_model(model_path, json_format=False)

model2 = CountSpace()
model2.load_model(model_path, json_format=False)

soyspacing 은 0 을 붙여쓰기 tag 로, 1 을 띄어쓰기 tag 로 이용합니다. None 은 불확실한 경우를 의미합니다.

soyspacing 은 각 글자들에 대하여 띄어쓰기 점수를 [-1, 1] 로 부여합니다. -1 은 반드시 붙여쓰기를 하는 경우이며 1 은 반드시 띄어쓰기를 하는 경우입니다. `nonspace_threshold` 는 이 점수 이하일 때에만 붙여쓰기 태그인 0 을 부여하고, `space_threshold` 이상일 때에만 띄어쓰기 태그인 1 을 부여합니다. 만약 띄어쓰기 점수가 `nonspace_threshold` < score < `space_threshold` 라면 None 을 부여합니다.

띄어쓰기에 이용할 features 가 학습데이터에서 `min_count` 보다 적게 등장하였다면 확실하지 않은 정보라는 의미입니다. `min_count` 이하로 등장한 features 는 tagging 에 이용하지 않습니다.

`force_abs_threshold` 는 띄어쓰기 교정을 가속화 하기 위한 parameter 입니다. 띄어쓰기를 위하여 점수를 초기화 할 때 띄어쓰기나 붙여쓰기 점수의 절대값이 이 점수보다 크다면 앞/뒤의 태그의 영향을 거의 받지 않을 것입니다. 초기 띄어쓰기 점수의 절대값이 `force_abs_threshold` 보다 크다면 sequential labeling 을 하지 않고 한 번에 모두 labeling 을 수행합니다.

`rules` 는 사용자가 임의로 강제할 수 있는 띄어쓰기 사용자 규칙입니다. 사용법은 아래에서 더 자세히 이야기합니다.

| Argument | Help |
| --- | --- |
| force_abs_threshold | 점수의 절대값이 이 수준 이상이면 최고점이 아니더라도 즉각 태깅 |
| nonspace_threshold  | 이 점수 이하일 때만 0으로 태깅 |
| space_threshold     | 이 점수 이상일 때만 1로 태깅 |
| min_count           | L, C, R 각각의 feature 빈도수가 min_count 이하이면 불확실한 정보로 판단, 띄어쓰기 계산 시 무시 |
| verbose             | iteration 마다 띄어쓰기가 어떻게 되고 있는지 확인 |
| rules               | 점수와 관계없이 반드시 태깅을 먼저 할 (chars, tags) |

In [5]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

라라랜드 영화 리뷰를 이용하여 학습한 모델이기 때문에 영화 리뷰에 등장할만한 표현들에 대해서는 띄어쓰기 교정이 이뤄집니다.

In [6]:
sent = '이건진짜좋은영화 라라랜드진짜좋은영화'

sent_corrected, tags = model.correct(
    sent,
    verbose = verbose,
    force_abs_threshold = ft,
    nonspace_threshold = nt,
    space_threshold = st,
    min_count = mc
)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 이건진짜좋은영화 라라랜드진짜좋은영화
after : 이건 진짜 좋은 영화 라라랜드진짜 좋은 영화


`라라랜드진짜` 부분에서는 띄어쓰기가 이뤄지지 않았는데, 이부분의 tags 가 `[None, None]` 입니다. 띄어쓰기 정보가 확실하지 않기 때문에 띄어쓰기를 하지 않았다는 의미입니다.

In [7]:
tags

[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 1, 0, 1]

사용자가 정의한 규칙을 입력할 수도 있습니다. 규칙 파일들은 여러 개를 입력할 수도 있습니다. 각 규칙파일은 tap separated 되어 있는 두 개의 column 입니다. 아래처럼 글자가 입력되고, 글자 앞, 뒤의 띄어쓰기 tags 로 이뤄져 있습니다. 101 은 `진짜` 라는 글자 앞에서 띄어쓰기를 하고, `진` 다음에는 붙여쓰며, `짜` 다음에는 반드시 띄어쓴다는 규칙입니다.

```
진짜 101
```

In [8]:
from soyspacing.countbase import RuleDict

rule_files = ['./ruledict']
rule_dict = RuleDict(rule_files)

sent_corrected, tags = model.correct(
    sent,
    verbose = verbose,
    force_abs_threshold = ft,
    nonspace_threshold = nt,
    space_threshold = st,
    min_count = mc,
    rules = rule_dict
)

print('before: %s' % sent)
print('after : %s' % sent_corrected)

before: 이건진짜좋은영화 라라랜드진짜좋은영화
after : 이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화


`correct` 함수에 `debug=True` 를 입력하면 띄어쓰기 교정이 되는 과정이 출력됩니다. rules 에 의하여 `진짜`가 포함된 두 부분이 강제로 교정이 되었으며, soyspacing 이 이용하는 (L, C, R) score 가 출력됩니다. 처음에 8 글자는 force tagging 이 이뤄졌으며, 두 개의 글자가 순차적으로 교정되었음을 확인할 수 있습니다.

In [10]:
model.correct(sent, debug=True, rules=rule_dict)

rule tagging (b=2, e=4), subchar=진짜, tags=(1, 0, 1)
rule tagging (b=12, e=14), subchar=진짜, tags=(1, 0, 1)
0: 이 (-1.000, 13)	lcr = (0.000, 0.000, -1.000)
1: 건 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
2: 진 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
3: 짜 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
4: 좋 (-1.000, 531)	lcr = (-1.000, 0.000, -1.000)
5: 은 (0.503, 1024)	lcr = (0.000, 0.377, 0.629)
6: 영 (-1.000, 1098)	lcr = (-1.000, -1.000, -1.000)
7: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
8: 라 (-0.998, 784)	lcr = (-1.000, -1.000, -0.994)
9: 라 (-0.931, 975)	lcr = (-0.882, -0.956, -0.955)
10: 랜 (-1.000, 941)	lcr = (-1.000, -1.000, 0.000)
11: 드 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
12: 진 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
13: 짜 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
14: 좋 (-1.000, 531)	lcr = (-1.000, 0.000, -1.000)
15: 은 (0.502, 1022)	lcr = (0.000, 0.377, 0.627)
16: 영 (-1.000, 1022)	lcr = (-1.000, -1.000, 0.000)
17: 화 (0.000, 0)	lcr = (0.000, 0.000, 0.000)
force tagging i=0, score=-1.000
force tagging

('이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화',
 [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1])